# LIGHTNING IN 15 MINUTES

https://lightning.ai/docs/pytorch/stable/starter/introduction.html

In [1]:
import os
from torch import optim, nn, utils, Tensor
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import lightning as L
import torch

In [2]:
# define any number of nn.Modules (or use your current ones)
encoder = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(64, 3))
decoder = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28))


In [3]:
class LitAutoEncoder(L.LightningModule):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = nn.functional.mse_loss(x_hat, x)
        # Logging to TensorBoard (if installed) by default
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [4]:
# init the autoencoder
autoencoder = LitAutoEncoder(encoder, decoder)

# setup data
dataset = MNIST(os.getcwd(), download=True, transform=ToTensor())
train_loader = utils.data.DataLoader(dataset)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:04<00:00, 2024322.28it/s]


Extracting /Users/haru256/repo/github.com/haru-256/graph-neural-network-sandbox/lightning/tutorial/getting-started/MNIST/raw/train-images-idx3-ubyte.gz to /Users/haru256/repo/github.com/haru-256/graph-neural-network-sandbox/lightning/tutorial/getting-started/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 158467.25it/s]


Extracting /Users/haru256/repo/github.com/haru-256/graph-neural-network-sandbox/lightning/tutorial/getting-started/MNIST/raw/train-labels-idx1-ubyte.gz to /Users/haru256/repo/github.com/haru-256/graph-neural-network-sandbox/lightning/tutorial/getting-started/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 1143140.50it/s]


Extracting /Users/haru256/repo/github.com/haru-256/graph-neural-network-sandbox/lightning/tutorial/getting-started/MNIST/raw/t10k-images-idx3-ubyte.gz to /Users/haru256/repo/github.com/haru-256/graph-neural-network-sandbox/lightning/tutorial/getting-started/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 1276246.32it/s]

Extracting /Users/haru256/repo/github.com/haru-256/graph-neural-network-sandbox/lightning/tutorial/getting-started/MNIST/raw/t10k-labels-idx1-ubyte.gz to /Users/haru256/repo/github.com/haru-256/graph-neural-network-sandbox/lightning/tutorial/getting-started/MNIST/raw



In [5]:
# train the model (hint: here are some helpful Trainer arguments for rapid idea iteration)
trainer = L.Trainer(limit_train_batches=100, max_epochs=1)
trainer.fit(model=autoencoder, train_dataloaders=train_loader)


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/haru256/repo/github.com/haru-256/graph-neural-network-sandbox/lightning/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
Missing logger folder: /Users/haru256/repo/github.com/haru-256/graph-neural-network-sandbox/lightning/tutorial/getting-started/lightning_logs

  | Name    | Type       | Params | Mode 
-----------------------------------------------
0 | encoder | Sequential | 50.4 K | train
1 | decoder 

Epoch 0: 100%|██████████| 100/100 [00:03<00:00, 25.51it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 100/100 [00:03<00:00, 25.47it/s, v_num=0]


In [6]:
# load checkpoint
checkpoint = "./lightning_logs/version_0/checkpoints/epoch=0-step=100.ckpt"
autoencoder = LitAutoEncoder.load_from_checkpoint(
    checkpoint, encoder=encoder, decoder=decoder
)

# choose your trained nn.Module
encoder = autoencoder.encoder
encoder.eval()

# embed 4 fake images!
fake_image_batch = torch.rand(4, 28 * 28, device=autoencoder.device)
embeddings = encoder(fake_image_batch)
print("⚡" * 20, "\nPredictions (4 image embeddings):\n", embeddings, "\n", "⚡" * 20)


⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡ 
Predictions (4 image embeddings):
 tensor([[ 0.1812,  0.7212, -1.0295],
        [ 0.2164,  0.7441, -1.2114],
        [ 0.1508,  0.5964, -0.8751],
        [ 0.3278,  0.8754, -1.2716]], device='mps:0',
       grad_fn=<LinearBackward0>) 
 ⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡
